In [6]:
import pandas as pd
from datetime_truncate import truncate
folder = 'SecurityData/SecurityData/'

def outliers_date(file, uid_col, mod_type='day', num_stddev=3):
    data = pd.read_csv(folder + file)
    data['date'] = pd.to_datetime(data['date'])
    mod_col = 'mod_' + mod_type
    norm_col = 'norm_' + mod_type
    data[mod_col] = data['date'].apply(lambda v: int((v - truncate(v, mod_type)).total_seconds()))
    data[norm_col] = (data[mod_col] - data[mod_col].mean())/data[mod_col].std()
    #lower = data[mod_col].quantile(out_percent)
    #upper = data[mod_col].quantile(1 - out_percent)
    outliers = data[abs(data[norm_col]) > num_stddev]
    return outliers.groupby(uid_col).size().sort_values(ascending=False)

In [11]:
import pandas as pd
from collections import defaultdict
from datetime_truncate import truncate
from operator import itemgetter
folder = 'SecurityData/SecurityData/'

def ranking(series_device=None, series_logon=None, series_email=None):
    '''Rank the most supsicous users based on their rank in the logon and email series.
    Return users that are the most suspicious based on how high they rank in these three series.
    Total score is like golf, where the lower score means more suspicious.
    Return series of names of most suspicious users.'''
    employee_data = pd.read_csv(folder + 'employee_info.csv')
    # Map of user ids to ranking
    uid2rank = defaultdict(int)
    # Add rank in device series to total rank
    if not series_device is None:
        for index, (uid, _) in enumerate(series_device.iteritems()):
            uid2rank[uid] += 100 - index
    # Add rank in logon series to total rank
    if not series_logon is None:
        for index, (uid, _) in enumerate(series_device.iteritems()):
            uid2rank[uid] += 100 - index
    # Add rank in email series to total rank
    if not series_email is None:
        for index, (email, _) in enumerate(series_device.iteritems()):
            # get uid from email
            uid = employee_data[employee_data['email'] == email]['user_id']
            uid2rank[uid] += 100 - index
    # List of suspicious users ordered by rank
    sorted_rank = sorted(uid2rank.items(), key=itemgetter(1), reverse=True)
    # Convert to dataframe
    rank_data = pd.DataFrame.from_records(sorted_rank, columns=['user_id', 'rank'])
    # Add rank to employee_data
    employee_rank_data = employee_data.merge(rank_data, how='left')
    # return sorted by rank descending
    return employee_rank_data.sort_values(by='rank', ascending=False)

ranking(outliers_date('device_info.csv', 'user'))
    

,month,employee_name,user_id,email,role,supervisor,rank
5670,10/1/17,Ayanna Yolanda Nielsen,AYN0502,Ayanna.Yolanda.Nielsen@dtaa.com,Technician,William Vernon Booth,100.0
3758,8/1/17,Ayanna Yolanda Nielsen,AYN0502,Ayanna.Yolanda.Nielsen@dtaa.com,Technician,William Vernon Booth,100.0
2796,7/1/17,Ayanna Yolanda Nielsen,AYN0502,Ayanna.Yolanda.Nielsen@dtaa.com,Technician,William Vernon Booth,100.0
855,5/1/17,Ayanna Yolanda Nielsen,AYN0502,Ayanna.Yolanda.Nielsen@dtaa.com,Technician,William Vernon Booth,100.0
1829,6/1/17,Ayanna Yolanda Nielsen,AYN0502,Ayanna.Yolanda.Nielsen@dtaa.com,Technician,William Vernon Booth,100.0
4717,9/1/17,Ayanna Yolanda Nielsen,AYN0502,Ayanna.Yolanda.Nielsen@dtaa.com,Technician,William Vernon Booth,100.0
6614,11/1/17,Ayanna Yolanda Nielsen,AYN0502,Ayanna.Yolanda.Nielsen@dtaa.com,Technician,William Vernon Booth,100.0
1279,6/1/17,Cyrus Jin Park,CJP0221,Cyrus.Jin.Park@dtaa.com,Physicist,Evelyn Xena Santana,99.0
6080,11/1/17,Cyrus Jin Park,CJP0221,Cyrus.Jin.Park@dtaa.com,Physicist,Evelyn Xena Santana,99.0
2250,7/1/17,Cyrus Jin Park,CJP0221,Cyrus.Jin.Park@dtaa.com,Physicist,Evelyn Xena Santana,99.0
